# ANALYSIS 1 

# TOP 5 Cuisines with the TOTAL NUMBER of RESTAURANTS offering them in every COUNTRY 

### ALL REQUIRED IMPORTS

In [1]:
import string
import csv, sys
from pandas import Series, DataFrame
import pandas as pd
import os
import glob
import fnmatch
import json
import nltk
from pprint import pprint

### Getting the current system path and defining the required path

In [18]:
directory = os.path.dirname(os.getcwd()) 
path =  directory + '/data/'
path
#directory

'/Users/sonalichaudhari/Desktop/FINAL_Python/data/'

### Retrieving all the Cuisines and the count of restaurants respective to each Country

In [ ]:
C={}

for subdir, dirs, files in os.walk(path):
    for file in files:
#fnmatch: Test whether the filename string matches the pattern of a json extension, returning True or False
            if fnmatch.fnmatch(file, '*.json'): 
                for f in glob.glob(subdir+'/*json'):
                    with open(f) as json_file:
                        d = json.load(json_file)
                        c = d['restaurant']['location']['country_name']
                        cu = d['restaurant']['cuisines'].split(',') #Use of nltk to split the string
                        if c not in C:
                            C[c]={}
                            for x in cu:
                                if x.strip() not in C[c]:
                                    C[c][x.strip()]= 1 #Use of nltk to eliminate blank spaces
                                else:
                                    C[c][x.strip()] += 1

### Loading the above data in the CSV file

In [107]:
cur_path = directory='/FINAL_Python/csv/'
header = ['COUNTRY','CUISINES','RESTAURANT COUNTS']
with open('ANA_1.csv', 'a') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)
        for key,values in C.items():
            z = {}
            z = dict(C[key])#(sorted(C[key].items(), key=lambda x: x[1], reverse=True))  
            for k,v in z.items():
                writer.writerow([key,k,v]) 

### Converting csv data into Dataframe

In [181]:
dataframe = pd.read_csv('/Users/sonalichaudhari/Desktop/FINAL_Python/ANA_1.csv').reset_index(0,drop=True)
dataframe.head(3)

,COUNTRY,CUISINES,RESTAURANT COUNTS
0,Italy,American,20
1,Italy,Cafe,100
2,Italy,Burger,120


### Getting only the the TOP 5 prominent cuisines for each Country 

In [223]:
p =dataframe.groupby('COUNTRY').apply(lambda x: x.sort_values('RESTAURANT COUNTS', ascending=False))# Sort using lambda function
s = p.groupby('COUNTRY').head(5)[['CUISINES','RESTAURANT COUNTS']]#.reset_index()
#s[['COUNTRY','CUISINES','RESTAURANT COUNTS']]
s

CUISINES  RESTAURANT COUNTS
COUNTRY                                                 
Australia      124  Modern Australian               3859
               126     Coffee and Tea               3260
               116               Cafe               3016
               136          Breakfast               1371
               125            Seafood               1360
Canada         49            Canadian               2872
               73              Burger               2413
               23            Bar Food               2175
               54             Seafood               2143
               40               Pizza               2123
India          429       North Indian               4857
               420            Chinese               3250
               412        Continental               2727
               409            Italian               2276
               404               Cafe               2133
Italy          9              Italian                200
               12               Pizza                180
               6             Desserts                140
               2               Burger                120
               13               Roman                100
Philippines    350           Japanese                 20
               367           Filipino                 17
               365              Asian                 16
               355            Seafood                 13
               353           American                 12
South Africa   343            Seafood                431
               324              Pizza                375
               323               Cafe                277
               338              Grill                219
               326             Burger                159
United Kingdom 285             Indian                616
               286            British                478
               283            Italian                418
               273           American                297
               296           Scottish                280
United States  162           American             172032
               217              Pizza              92037
               212           Sandwich              65455
               189             Burger              64157
               220            Seafood              60372

In [227]:
g = p.groupby('COUNTRY').head(5)[['CUISINES','RESTAURANT COUNTS']].reset_index()
import matplotlib.pyplot as plt
ax = g[['CUISINES','RESTAURANT COUNTS']].plot(kind='bar', title ="V comp", figsize=(15, 10), legend=True, fontsize=12)
ax.set_xlabel("Hour", fontsize=12)
ax.set_ylabel("V", fontsize=12)
#plt.show()

In [236]:
g = p.groupby('COUNTRY').head(5)[['CUISINES','RESTAURANT COUNTS']].reset_index()[['COUNTRY','CUISINES','RESTAURANT COUNTS']]
g

,COUNTRY,CUISINES,RESTAURANT COUNTS
0,Australia,Modern Australian,3859
1,Australia,Coffee and Tea,3260
2,Australia,Cafe,3016
3,Australia,Breakfast,1371
4,Australia,Seafood,1360
5,Canada,Canadian,2872
6,Canada,Burger,2413
7,Canada,Bar Food,2175
8,Canada,Seafood,2143
9,Canada,Pizza,2123


In [246]:
#for c in g['COUNTRY']:
   #print(c)

In [258]:
for x in g.loc[[2]]:
    print(x)

COUNTRY
CUISINES
RESTAURANT COUNTS


In [234]:
data = [[5., 25., 50., 20.],
  [4., 23., 51., 17.],
  [6., 22., 52., 19.]]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = [[5., 25., 50., 20.],
  [4., 23., 51., 17.],
  [6., 22., 52., 19.]]

X = np.arange(4)
plt.bar(X + 0.00, data[0], color = 'b', width = 0.25)
plt.bar(X + 0.25, data[1], color = 'g', width = 0.25)
plt.bar(X + 0.50, data[2], color = 'r', width = 0.25)

plt.show()